In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pydot

In [2]:
def find_entropy(df):
    target = df.keys()[-1]
    values,counts = np.unique(df[target],return_counts=True)#target'daki ifadeleri adetleriyle beraber al 
    entropy = 0
    for i in range(len(values)):
        fraction =counts[i]/len(df[target]) #target'daki bir değerin sayısı/df uzunluğu
        entropy += -fraction*np.log2(fraction) #target değerine göre entropy hesabı
    return entropy

In [3]:
def find_entropy_arg(df,arg):
    target = df.keys()[-1]
    value_1,counts_1 = np.unique(df[arg],return_counts=True) #arg sütununu değerlerini sayılarıyla beraber al
    entropy=0
    for i in range(len(counts_1)):
        frac=counts_1[i]/len(df) #bir değerin adedi/df uzunluğu
        table=get_subtable(df,arg,value_1[i]) #her bir değere göre subtable oluştur
        value_2,counts_2 = np.unique(table[target],return_counts=True) #subtable'da target değerlerini adetleriyle beraber al
        ent=0
        for j in counts_2:
            frac2 = j/len(table) #target'daki bir değerin sayısı/subtable uzunluğu
            ent += frac2*np.log2(frac2) #target'daki bir değer için entropy hesabı
            
        entropy+=frac*ent #df'deki bir sütun için entropy hesabı
    return -entropy

In [4]:
def find_node(df):
    columns=df.keys()[:-1]
    gain_list = []
    for column in columns:
        gain_list.append(find_entropy(df)-find_entropy_arg(df,column)) #her bir sütün için kazanç hesaplayıp listeye ekleme
        
    index=np.argmax(gain_list)#max kazanç değerinin indexi
    node=columns[index] # kazancı max olan sütunun node olarak return edilmesi
    
    return node

In [5]:
def get_subtable(df, node,value):
    # node sütununda value değerine sahip satırların table olarak atanması
    table=df[df[node] == value].reset_index(drop=True)
    return table

In [6]:
def buildTree(df,tree=None): 
    target = df.keys()[-1] #target sütunu belirle
    
    node = find_node(df) #dallanacak sütunu bul
    
    sub_values = set(df[node]) #sütunun değerlerini al
        
    if tree is None: #fonksiyon ilk defa çağrılıyorsa ağaç oluştur                    
        tree={}
        tree[node] = {}
        
    for value in sub_values:
        #düğümün değerine göre subtable oluştur
        subtable = get_subtable(df,node,value) 
        
        #subtable'ın target değerlerini sayılarıyla beraber al     
        subtable_value,counts = np.unique(subtable[target],return_counts=True)          
        
        if len(counts)==1:#counts 1 ise target'daki değerler aynıdır 
            tree[node][value] = subtable_value[0]    #dallanmayı sonlandır                                                
        else:        
            tree[node][value] = buildTree(subtable) #target'da tek bir değer olmadığı için dallanmayı sürdür
                   
    return tree

In [7]:
#görselleştirme kısmı
def draw(parent_name, child_name):
    edge = pydot.Edge(parent_name, child_name)
    graph.add_edge(edge)
def visit(node, parent=None):
    for k,v in node.items():
        if(type(k)!=dict):
            k=str(k)
        if(type(v)!=dict):
            v=str(v)
        
        if isinstance(v, dict):
            if parent:
                draw(parent, k)
            visit(v, k)
        else:
            draw(parent, k)
            draw(k, k+'_'+v)
      


In [8]:
df = pd.read_csv('netflix_titles.csv')
y=df["rating"]

In [9]:
#kullanılmayan sütnlar silindi
dropList = ["description","date_added","show_id","title","cast","rating"]
df.drop(dropList, axis=1, inplace=True)

In [10]:
#rating sütununun dataframein en sonunda yer alması sağlandı
df["rating"]=y

In [11]:
#nan değerler veri setinden silindi
df=df.dropna()
df = df.reset_index(drop=True)

In [12]:
#veri train ve test olarak ikiye bölünür
X_train, X_test= train_test_split(df, test_size=0.3, random_state=42)

In [13]:
#X_train için karar ağacı oluşturulur
tree=buildTree(X_train)

In [14]:
#ağaç görselleştirilir
graph = pydot.Dot(graph_type='graph')
visit(tree)
graph.write_png('tree_graph.png',encoding='utf-8')

In [15]:
#datasetin tamamıyla yapılan png çok büyük ve anlaşılması güç
#görselleştirme fonksiyonun işleyişini göstermek için küçük bir ağaç çizdirildi
small_tree=buildTree(X_train.head(100))
graph = pydot.Dot(graph_type='graph')
visit(small_tree)
graph.write_png('small_tree_graph.png',encoding='utf-8')